In [0]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data

mnist=input_data.read_data_sets("./mnist/data/",one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/datas

In [0]:
# 데이터를 넣을 공간인 placeholder 구성하기.
# 데이터 개수는 값을 정해주지 않고 열어두는 의미로 None
# conv2d는 2d 데이터를 받으므로 2d 형태로 28*28,
# 흑백 이미지이므로 channel = 1
X=tf.placeholder(tf.float32,[None,28,28,1])
# 데이터 개수는 오픈해두는 의미로 None
# label 즉, target값이 0~9까지 숫자 이므로 10개의 클래스
Y=tf.placeholder(tf.float32,[None,10])



keep_prob=tf.placeholder(tf.float32)

In [0]:
# 첫 번째 레이어의 필터를 랜덤 정규화(표준편차 0.01)한 값으로 넣기
# 필터 : 5*5 사이즈, input이 흑백인 채널이므로 채널수는 1, 20개의 필터를 만들겠다.
W1=tf.Variable(tf.random_normal([5,5,1,20],stddev=0.01))
# convolution layer 1 만들기
# layer1은 입력으로 X를 받고 필터 W1을 사용하고
# stride = [샘플수, 세로 길이, 가로 길이, 채널 수]
# padding='SAME' : 추가 패딩은 넣지 않음(사이즈 그대로)
L1=tf.nn.conv2d(X,W1,strides=[1,1,1,1],padding="SAME")
L1=tf.nn.relu(L1)

print(L1)

Tensor("Relu:0", shape=(?, 28, 28, 20), dtype=float32)


In [0]:
# 첫 번째 convolution layer 에 Pooling Layer(크기를 줄여주는 레이어) 적용하기
# 입력 :이전 Layer
# ksize = [샘플 수, 세로 사이즈, 가로 사이즈, 채널 수]
# :pooling layer에서의 필터의 사이즈를 의미
# strides = [샘플 수, 세로 스트라이드, 가로 스트라이드, 채널 수]
# :polling layer에서의 필터의 움직임을 의미
# strides 는 가로 세로 각각 1로 하자.
# padding = 'SAME' : 사이즈 유지
L1=tf.nn.max_pool(L1,ksize=[1,4,4,1],strides=[1,4,4,1],padding="SAME")

In [0]:
print(L1)

Tensor("MaxPool:0", shape=(?, 7, 7, 20), dtype=float32)


In [0]:
W2=tf.Variable(tf.random_normal([5,5,20,53],stddev=0.01))

L2=tf.nn.conv2d(L1,W2,strides=[1,1,1,1],padding="SAME")
L2=tf.nn.relu(L2)

print(L2)

Tensor("Relu_1:0", shape=(?, 7, 7, 53), dtype=float32)


In [0]:
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1,2,2,1], padding='SAME')
print(L2)

Tensor("MaxPool_1:0", shape=(?, 4, 4, 53), dtype=float32)


In [0]:
# 세 번째 레이어(Fully Connected Layer)
# reshape와 행렬곱을 적용함

# 20개의 필터를 만들겠다.
# 필터를 랜덤 정규화(표준편차 0.01)한 값으로 넣기
# 1차원으로 펴주는 flatten 작용이 들어가므로 [4, 4, 53, 20]으로 들어가지 않는다.
# 앞단 필터의 가중치의 갯수, 현재 적용하고자하는 필터의 갯수
W3=tf.Variable(tf.random_normal([4*4*53,20],stddev=0.01))
# reshape 과정이 들어간다.
# -1은 몇개의 데이터가 들어올지 모르므로 열어두는 역할
# 뒤에 값은 앞단 필터의 가중치의 갯수
L3=tf.reshape(L2,[-1,4*4*53])
#행렬곱
L3=tf.matmul(L3,W3)
# 엑티베이션 펑션으로 relu 사용
L3=tf.nn.relu(L3)
# dropout 을 통해 과적합을 줄여주자.
L3 = tf.nn.dropout(L3,keep_prob)
print(L3)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Tensor("dropout/mul_1:0", shape=(?, 20), dtype=float32)


In [0]:
# 마지막 레이어(Flatten Layer)
# 필터를 랜덤 정규화(표준편차 0.01)한 값으로 넣기
# 앞단 필터의 갯수, 현재 적용하고자 하는 필터수를 리스트로 넣어준다.
# 현재 얻고자 하는 필터수는 10개의 숫자 값이므로 10
W4=tf.Variable(tf.random_normal([20,10],stddev=0.01))

model=tf.matmul(L3,W4)
print(model)

Tensor("MatMul_1:0", shape=(?, 10), dtype=float32)


In [0]:
loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer=tf.train.AdamOptimizer(0.001).minimize(loss)

In [0]:
sess=tf.Session()
init=tf.global_variables_initializer()
sess.run(init)

# batch_size를 500으로 설정해보자.
batch_size=500
total_batch=int(mnist.train.num_examples/batch_size)

for epoch in range(0,3):
  %%time
  total_loss = 0
  for i in range(total_batch):
    batch_xs, batch_ys = mnist.train.next_batch(batch_size)
    # 이미지 데이터를 CNN 모델을 위한 자료형태인 [28 28 1] 의 형태로 재구성합니다.
    batch_xs = batch_xs.reshape(-1, 28, 28, 1)
    _, loss_val = sess.run([optimizer, loss],
                            feed_dict={X: batch_xs,Y: batch_ys, keep_prob: 0.7})

    total_loss += loss_val
  print('Epoch:', '%04d' % (epoch + 1),
        'Avg. loss =', '{:.3f}'.format(total_loss / total_batch))
print('최적화 완료!')
  

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.44 µs
Epoch: 0001 Avg. loss = 1.474
CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.68 µs
Epoch: 0002 Avg. loss = 0.701
CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.44 µs
Epoch: 0003 Avg. loss = 0.505
최적화 완료!
